In [5]:
#192.168.99.100# Loading the required libraries (Pandas, scikit-learn)
#import graphviz 
import pandas as pd
import numpy as np
import datetime
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn import tree
from pymongo import MongoClient

In [6]:
# Load dataset

path = "D:/OE0t11824/ATT/mongo_dump/vijay/SOTrainingData.csv"
names = ['cpu', 'memory', 'network', 'diskSpace', 'numOfUser','noOfConn', 'responseTime','flaps', 'result','VNFSource']
rawdata = pd.read_csv(path, names=names)

client = MongoClient(host='localhost:27017')
db = client.dcomp
#rawdata_collection = db.rawdata
predictdata_collection = db.predictdata_collection
datacount_collection = db.datacount_collection
#rawdata = pd.DataFrame(list(rawdata_collection.find({},{"_id":0})))

#insert/update dates to raw data
dates=[]
lag =0
for row in rawdata['result']: 
    dates.append(datetime.datetime.now() + datetime.timedelta(0,10+lag))
    lag = lag + 50         
rawdata['date'] = dates

In [7]:

# Split-out validation dataset
array = rawdata.values
X = array[:,0:8]
Y = array[:,8]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)


In [8]:
# make prediction (KNN)
#knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model = KNeighborsClassifier(n_neighbors=3)
#print(knn_model)
knn_model.fit(X_train, Y_train)
knn_predictions = knn_model.predict(X_validation)
print(knn_predictions) 
print(Y_validation)
print(accuracy_score(Y_validation,knn_predictions))
accuracy = accuracy_score(Y_validation,knn_predictions)

['Pass' 'Disk Full' 'Response' 'Pass' 'High Memory' 'CPU Memory' 'Pass'
 'Pass' 'Pass' 'Pass' 'High Memory' 'Pass' 'Disk Full' 'CPU Memory'
 'High Memory' 'Pass' 'Service Failure' 'CPU Memory' 'Service Failure'
 'CPU Memory' 'Connections' 'Pass']
['Pass' 'Disk Full' 'Response' 'Pass' 'High Memory' 'CPU Memory' 'Pass'
 'Pass' 'Pass' 'Pass' 'High Memory' 'Pass' 'Disk Full' 'CPU Memory'
 'High Memory' 'Pass' 'Service Failure' 'CPU Memory' 'Service Failure'
 'CPU Memory' 'Connections' 'Pass']
1.0


In [10]:
# make prediction on unseen data
rawdata["id"] = rawdata.index + 1
predictdata = rawdata
predictdata['prediction'] = knn_model.predict(X)
#predictdata['date'] = datetime.datetime.now()
predictdata['date'] = rawdata['date']
#predictdata['predictdate'] = datetime.datetime.now()

In [11]:
# count based on Prediction status
passcount = len(predictdata[predictdata['prediction'] == 'Pass'])
sfcount = len(predictdata[predictdata['prediction'] == 'Service Faliure'])
cpumemorycount = len(predictdata[predictdata['prediction'] == 'CPU Memory'])
memoryleakcount = len(predictdata[predictdata['prediction'] == 'Memory Leak'])
responsecount = len(predictdata[predictdata['prediction'] == 'Response'])
memorycount = len(predictdata[predictdata['prediction'] == 'High Memory'])
connectioncount = len(predictdata[predictdata['prediction'] == 'Connections'])
diskfullcount = len(predictdata[predictdata['prediction'] == 'Disk Full'])
slowcount=cpumemorycount + responsecount+memorycount+connectioncount
failcount = sfcount+memoryleakcount+diskfullcount
accuracy = accuracy *100

In [13]:
cnt_dict = {}
cnt_dict['pass']= passcount
cnt_dict['serviceFaliure']=sfcount
cnt_dict['cpuMemory']=cpumemorycount
cnt_dict['memoryLeak']=memoryleakcount
cnt_dict['response']=responsecount
cnt_dict['highMemory']=memorycount
cnt_dict['connections']=connectioncount
cnt_dict['diskFull']=diskfullcount
cnt_dict['slow']=slowcount
cnt_dict['fail']=failcount
cnt_dict['accuracy']=accuracy

In [14]:
columns=list(cnt_dict.keys())
values = list(cnt_dict.values())
arr_len = len(values)
df_cnt=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
df_cnt['id'] = df_cnt.index + 1
#print(rawdata)
#print(predictdata)
#print(df_cnt)

In [15]:
db.predictdata_collection.delete_many({})
db.datacount_collection.delete_many({})
db.predictdata_collection.insert_many(predictdata.to_dict('records'))
db.datacount_collection.insert_many(df_cnt.to_dict('records'))